In [ ]:
import lib._util.visualplot as vp
import lib._util.fileproc as fp

# Feature encoding
from lib._class.DFOneHotEncoder import DFOneHotEncoder

# Feature scaling
from lib._class.DFMinMaxScaler import DFMinMaxScaler

# Feature extraction
from lib._class.DFIvis import DFIvis

In [ ]:
import pandas as pd
pd.set_option('display.max_columns', 100)

import numpy as np

# Plotly
import plotly.express as px
import plotly.graph_objects as go
from plotly.colors import DEFAULT_PLOTLY_COLORS

# Scikit-Learn
from sklearn.pipeline import Pipeline
from sklearn.metrics import classification_report, confusion_matrix

# Tensorflow
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, BatchNormalization, Dropout, Activation
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.metrics import Precision, Recall, AUC, CategoricalAccuracy
from tensorflow.keras.callbacks import ReduceLROnPlateau, EarlyStopping
from tensorflow.keras.constraints import max_norm

# Useful Functions

In [ ]:
COMPANY_CODE      = 'MAG'
TARGET            = 'target_d4'
SOURCE_PATH_TRANS = f'resources/output/eda_trans/file/{COMPANY_CODE}/'
OUT_PATH_GRAPH    = f'resources/output/ann_digit_trans/graph/{COMPANY_CODE}/'
OUT_PATH_FILE     = f'resources/output/ann_digit_trans/file/{COMPANY_CODE}/'

# Phase 1 - Feature Loading
- Load digit frequency

In [ ]:
def load_feature(filename):
    source_file = f'{SOURCE_PATH_TRANS}{filename}'
    df_chunks   = pd.read_csv(source_file, sep=';',
                              parse_dates=['draw_date'],
                              date_parser=lambda x: pd.to_datetime(x, format='%Y-%m-%d'),
                              chunksize=50_000)
    return pd.concat(df_chunks)

In [ ]:
feature_df = load_feature(f'{COMPANY_CODE} - digit_frequency.csv')

vp.faststat(feature_df)

# Phase 2 - Target Loading
- Create target label

In [ ]:
def load_target(filename):
    source_file = f'{SOURCE_PATH_TRANS}{filename}'
    df_chunks   = pd.read_csv(source_file, sep=';',
                              dtype={x: str for x in ['1st', '2nd', '3rd'] + \
                                                     [f'Sp{x +1}' for x in range(10)] + \
                                                     [f'Cons{x +1}' for x in range(10)]},
                              parse_dates=['draw_date'],
                              date_parser=lambda x: pd.to_datetime(x, format='%Y-%m-%d'),
                              chunksize=50_000)
    return pd.concat(df_chunks)

In [ ]:
target_df = load_target(f'{COMPANY_CODE} - transactions.csv')

vp.faststat(target_df)

In [ ]:
# Take target from following period
target_df['target'] = target_df['1st'].shift(-1)

# Split price numbers & target into digits
categories = ['1st', '2nd', '3rd'] + [f'Sp{x +1}' for x in range(10)] + [f'Cons{x +1}' for x in range(10)]
for column in categories + ['target']:
    for index in [x for x in range(4)]:
        new_column = f'{column}_d{4 - index}'
        target_df[new_column] = target_df[column].apply(lambda x: x[index] if x == x else x)

target_df.head()

In [ ]:
target_df.drop(columns=categories + ['target'], inplace=True)

# Phase 3 - Dataset
- Map target label to features

In [ ]:
feature_df.shape, target_df.shape

In [ ]:
data_df = feature_df.merge(target_df, on=['draw_date', 'draw_period'], how='inner')

vp.faststat(data_df)

In [ ]:
data_df.dropna(inplace=True)

columns = [x for x in data_df.columns if any([x.startswith(y) for y in categories + ['target']])]
data_df[columns] = data_df[columns].astype(int)

# Target distribution
print('Full dataset:')
vp.value_count(data_df, TARGET)

In [ ]:
def balanced_target(df, target, n_remain, excludes=[], random_state=None):
    np.random.seed(random_state)
    
    dfs = []
    for target_label in np.unique(df[target]):
        indexes = df[df[target] == target_label].index
        indexes = [x for x in indexes if x not in excludes]
        
        choices = np.random.choice(indexes, size=n_remain, replace=False)
        dfs.append(df[df.index.isin(choices)].copy())
        
    return pd.concat(dfs)

In [ ]:
# Split train dataset with balanced target label
train_df = balanced_target(data_df, target=TARGET, n_remain=500, random_state=10000)

# Remaining goes to test dataset
used_indexes = list(train_df.index)
test_df      = data_df[~data_df.index.isin(used_indexes)].copy()

# Shuffle dataset
train_df = train_df.sample(frac=1, random_state=0).reset_index(drop=True)
test_df  = test_df.sample(frac=1, random_state=0).reset_index(drop=True)

train_df.shape, test_df.shape

In [ ]:
print('Train dataset:')
vp.value_count(train_df, TARGET)

print('\nTest dataset:')
vp.value_count(test_df, TARGET)

###### Bar

In [ ]:
def sampling_period(df, title):
    sample_df = df.copy()
    sample_df['year_month'] = sample_df['draw_date'].dt.to_period('M').astype(str)
    sample_df = sample_df.groupby(['dataset', 'year_month']).agg(
        count=('year_month', 'count')
    ).reset_index()
    
    fig = px.bar(sample_df, x='year_month', y='count', facet_row='dataset')
    fig.update_layout(title=title)
    vp.generate_plot(fig,
                     out_path=OUT_PATH_GRAPH,
                     out_filename=title)

In [ ]:
train_df['dataset'] = 'train'
test_df['dataset']  = 'test'

sampling_period(pd.concat([train_df, test_df]),
                title='Phase 2 - Bar - Draw Date (Sample)')

# Phase 4 - Feature Extraction
- Separate dataset to features & target
- Feature encoding
- Ivis dimension reduction

In [ ]:
def feature_target_split(df):
    X = df[[x for x in train_df.columns
            if all([not x.startswith(y) for y in ['draw_date', 'draw_period', 'dataset', 'target']])]]
    y = df[TARGET]
    
    return X, y

In [ ]:
# Separate features & target
X_train, y_train = feature_target_split(train_df)
X_test,  y_test  = feature_target_split(test_df)

print('Train dataset:')
print(X_train.shape, y_train.shape)

print('\nTest dataset:')
print(X_test.shape, y_test.shape)

In [ ]:
# Binary crossentropy target
target_onehot_encoder = DFOneHotEncoder(dtype='byte')
y_train_binary = target_onehot_encoder.fit_transform(y_train.to_frame())

# Sparse categorical crossentropy target
y_train_sparse = y_train.copy()

y_train_binary.shape, y_train_sparse.shape

In [ ]:
# Feature extraction
onehot_columns = [x for x in X_train.columns if any([x.startswith(y) for y in categories])]
onehot_encoder = DFOneHotEncoder(columns=onehot_columns, dtype='byte', drop='first')

minmax_scaler  = DFMinMaxScaler(columns=[x for x in X_train.columns if x not in onehot_columns])

# TODO - define own model to prevent overfitting & explosive stacked_triplets_loss
ivis_binary    = DFIvis(embedding_dims=2, epochs=100,
                        k=15, n_epochs_without_progress=10, model='szubert',
                        supervision_weight=1, supervision_metric='binary_crossentropy', distance='pn')

ivis_sparse    = DFIvis(embedding_dims=2, epochs=100,
                        k=15, n_epochs_without_progress=10, model='szubert',
                        supervision_weight=1, supervision_metric='sparse_categorical_crossentropy', distance='pn')

# Binary crossentropy
steps = [
    ('onehot_encoder', onehot_encoder),
    ('minmax_scaler', minmax_scaler),
    ('ivis_binary', ivis_binary),
]
ivis_binary_pipeline = Pipeline(steps, verbose=True)

X_train_binary = ivis_binary_pipeline.fit_transform(X_train, y_train_binary)
X_test_binary  = ivis_binary_pipeline.transform(X_test)

X_train_binary.shape, X_test_binary.shape

In [ ]:
# Sparse categorical crossentropy
steps = [
    ('onehot_encoder', onehot_encoder),
    ('minmax_scaler', minmax_scaler),
    ('ivis_sparse', ivis_sparse),
]
ivis_sparse_pipeline = Pipeline(steps, verbose=True)

X_train_sparse = ivis_sparse_pipeline.fit_transform(X_train, y_train_sparse)
X_test_sparse  = ivis_sparse_pipeline.transform(X_test)

X_train_sparse.shape, X_test_sparse.shape

###### Line

In [ ]:
names      = ['binary_loss', 'sparse_loss']
losses     = [ivis_binary.model.loss_history_, ivis_sparse.model.loss_history_]
max_length = max([len(x) for x in losses])

tmp_df = pd.DataFrame([x+1 for x in range(max_length)], columns=['epoch'])
for index, loss in enumerate(losses):
    loss += [np.nan] * (max_length - len(loss))
    tmp_df[names[index]] = loss

vp.line(tmp_df,
        xy_tuples=[('epoch', x) for x in names],
        title='Phase 3 - Line - Ivis Loss',
        out_path=OUT_PATH_GRAPH)

del tmp_df

###### Scatter

In [ ]:
title  = 'Phase 3 - Scatter - Ivis - Binary'
tmp_df = pd.concat([X_train_binary, pd.Series(np.argmax(y_train_binary.values, axis=1), name=TARGET).astype(str)], axis=1)

fig = px.scatter(
    tmp_df.sort_values(by=TARGET),
    x='ivis_0',
    y='ivis_1',
    color=TARGET,
    marginal_x='histogram'
)
fig.update_layout(title=title)

vp.generate_plot(fig,
                 out_path=OUT_PATH_GRAPH,
                 out_filename=title)

del tmp_df

In [ ]:
title  = 'Phase 3 - Scatter - Ivis - Sparse Categorical'
tmp_df = pd.concat([X_train_sparse, y_train_sparse.astype(str)], axis=1)

fig = px.scatter(
    tmp_df.sort_values(by=TARGET),
    x='ivis_0',
    y='ivis_1',
    color=TARGET,
    marginal_x='histogram'
)
fig.update_layout(title=title)

vp.generate_plot(fig,
                 out_path=OUT_PATH_GRAPH,
                 out_filename=title)

del tmp_df

In [ ]:
title  = 'Phase 3 - Scatter - Ivis - Binary (Test dataset)'
tmp_df = pd.concat([X_test_binary, y_test.astype(str)], axis=1)

fig = px.scatter(
    tmp_df.sort_values(by=TARGET),
    x='ivis_0',
    y='ivis_1',
    color=TARGET,
    marginal_x='histogram'
)
fig.update_layout(title=title)

vp.generate_plot(fig,
                 out_path=OUT_PATH_GRAPH,
                 out_filename=title)

del tmp_df

In [ ]:
title  = 'Phase 3 - Scatter - Ivis - Sparse Categorical (Test dataset)'
tmp_df = pd.concat([X_test_sparse, y_test.astype(str)], axis=1)

fig = px.scatter(
    tmp_df.sort_values(by=TARGET),
    x='ivis_0',
    y='ivis_1',
    color=TARGET,
    marginal_x='histogram'
)
fig.update_layout(title=title)

vp.generate_plot(fig,
                 out_path=OUT_PATH_GRAPH,
                 out_filename=title)

del tmp_df

# Phase 5 - Classification
- Binary crossentropy features

In [ ]:
# Split train & validation dataset with balanced target label
train_df = balanced_target(pd.concat([X_train_binary, y_train], axis=1),
                           target=TARGET, n_remain=350, random_state=10000)
valid_df = balanced_target(pd.concat([X_train_binary, y_train], axis=1),
                           target=TARGET, n_remain=150, random_state=10000, excludes=train_df.index)

# Shuffle dataset
train_df = train_df.sample(frac=1, random_state=0).reset_index(drop=True)
valid_df = valid_df.sample(frac=1, random_state=0).reset_index(drop=True)

train_df.shape, valid_df.shape

In [ ]:
print('Train dataset:')
vp.value_count(train_df, TARGET)

print('\nValidate dataset:')
vp.value_count(valid_df, TARGET)

In [ ]:
# Separate features & target
X_train_binary, y_train_binary = feature_target_split(train_df)
X_valid_binary, y_valid_binary = feature_target_split(valid_df)

# Categorical crossentropy target
y_train_binary = target_onehot_encoder.transform(y_train_binary.to_frame())
y_valid_binary = target_onehot_encoder.transform(y_valid_binary.to_frame())
y_test_binary  = target_onehot_encoder.transform(y_test.to_frame())

print('Train dataset:')
print(X_train_binary.shape, y_train_binary.shape)

print('\nValidate dataset:')
print(X_valid_binary.shape, y_valid_binary.shape)

In [ ]:
# Feature scaling
minmax_scaler  = DFMinMaxScaler()
X_train_binary = minmax_scaler.fit_transform(X_train_binary)
X_valid_binary = minmax_scaler.transform(X_valid_binary)
X_test_binary  = minmax_scaler.transform(X_test_binary)

X_train_binary.shape, X_valid_binary.shape, X_test_binary.shape

In [ ]:
def compile_model(X):
    model = Sequential()
    model.add(Dense(128, input_dim=X.shape[1],
                    kernel_initializer='he_uniform',
                    kernel_constraint=max_norm(5),
                    use_bias=False))
    model.add(BatchNormalization(scale=False,
                                 renorm=True,
                                 renorm_clipping={ 'rmax': 1, 'rmin': 0, 'dmax': 0 }))
    model.add(Activation('relu'))
    model.add(Dropout(rate=.2))
    
    model.add(Dense(64,
                    kernel_initializer='he_uniform',
                    kernel_constraint=max_norm(5),
                    use_bias=False))
    model.add(BatchNormalization(scale=False,
                                 renorm=True,
                                 renorm_clipping={ 'rmax': 1, 'rmin': 0, 'dmax': 0 }))
    model.add(Activation('relu'))
    model.add(Dropout(rate=.2))
    
    model.add(Dense(10, activation='softmax'))
    
    # Referecence: https://www.tensorflow.org/tutorials/structured_data/imbalanced_data
    metrics = [
        CategoricalAccuracy(name='categorical_accuracy'),
        AUC(name='auc'),
        Precision(name='precision'),
        Recall(name='recall'),
    ]
    model.compile(loss='categorical_crossentropy',
                  optimizer=Adam(lr=.001, epsilon=.00001),
                  metrics=metrics)
    
    return model

In [ ]:
# Reference: https://machinelearningmastery.com/understand-the-dynamics-of-learning-rate-on-deep-learning-neural-networks/
lrate = ReduceLROnPlateau(monitor='val_auc', factor=0.95, patience=15)

# Reference: https://machinelearningmastery.com/how-to-stop-training-deep-neural-networks-at-the-right-time-using-early-stopping/
es = EarlyStopping(monitor='val_auc', mode='max', verbose=1, patience=30, restore_best_weights=True)

model   = compile_model(X_train_binary)
history = model.fit(X_train_binary, y_train_binary,
                    validation_data=(X_valid_binary, y_valid_binary),
                    epochs=100,
                    callbacks=[lrate, es],
                    batch_size=32)

In [ ]:
def eval_classif(y_true, y_pred):
    cofmat_df = pd.DataFrame(confusion_matrix(y_true, y_pred))
    cofmat_df.index.name   = 'True'
    cofmat_df.columns.name = 'Pred'

    print(cofmat_df)
    print()
    print(classification_report(y_true, y_pred, digits=5))

In [ ]:
# Test set evaluation
eval_classif(
    np.argmax(y_test_binary.values, axis=1),
    np.argmax(model.predict(X_test_binary), axis=1)
)

In [ ]:
# Train set evaluation
eval_classif(
    np.argmax(y_train_binary.values, axis=1),
    np.argmax(model.predict(X_train_binary), axis=1)
)

In [ ]:
# Validation set evaluation
eval_classif(
    np.argmax(y_valid_binary.values, axis=1),
    np.argmax(model.predict(X_valid_binary), axis=1)
)

###### Line

In [ ]:
# Learning Rate
data = []
data.append(go.Scatter(
    y=history.history['lr'],
    mode='lines',
    name='LR',
    marker={'color': DEFAULT_PLOTLY_COLORS[-1]},
))
fig1 = go.Figure(data=data)

# Loss
data = []
data.append(go.Scatter(
    y=history.history['loss'],
    mode='lines',
    name='loss',
    marker={'color': DEFAULT_PLOTLY_COLORS[0]},
    legendgroup='train',
))
data.append(go.Scatter(
    y=history.history['val_loss'],
    mode='lines',
    name='val_loss',
    marker={'color': DEFAULT_PLOTLY_COLORS[1]},
    legendgroup='validate',
))
fig2 = go.Figure(data=data)

# Accuracy
data = []
data.append(go.Scattergl(
    y=history.history['categorical_accuracy'],
    mode='lines',
    name='accuracy',
    marker={'color': DEFAULT_PLOTLY_COLORS[0]},
    legendgroup='train',
))
data.append(go.Scattergl(
    y=history.history['val_categorical_accuracy'],
    mode='lines',
    name='val_accuracy',
    marker={'color': DEFAULT_PLOTLY_COLORS[1]},
    legendgroup='validate',
))
fig3 = go.Figure(data=data)

# AUC
data = []
data.append(go.Scattergl(
    y=history.history['auc'],
    mode='lines',
    name='auc',
    marker={'color': DEFAULT_PLOTLY_COLORS[0]},
    legendgroup='train',
))
data.append(go.Scattergl(
    y=history.history['val_auc'],
    mode='lines',
    name='val_auc',
    marker={'color': DEFAULT_PLOTLY_COLORS[1]},
    legendgroup='validate',
))
fig4 = go.Figure(data=data)

# Precision
data = []
data.append(go.Scatter(
    y=history.history['precision'],
    mode='lines',
    name='precision',
    marker={'color': DEFAULT_PLOTLY_COLORS[0]},
    legendgroup='train',
))
data.append(go.Scatter(
    y=history.history['val_precision'],
    mode='lines',
    name='val_precision',
    marker={'color': DEFAULT_PLOTLY_COLORS[1]},
    legendgroup='validate',
))
fig5 = go.Figure(data=data)

# Recall
data = []
data.append(go.Scatter(
    y=history.history['recall'],
    mode='lines',
    name='recall',
    marker={'color': DEFAULT_PLOTLY_COLORS[0]},
    legendgroup='train',
))
data.append(go.Scatter(
    y=history.history['val_recall'],
    mode='lines',
    name='val_recall',
    marker={'color': DEFAULT_PLOTLY_COLORS[1]},
    legendgroup='validate',
))
fig6 = go.Figure(data=data)

data_groups = [fig1['data'], fig2['data'], fig3['data'], fig4['data'], fig5['data'], fig6['data']]
vp.datagroups_subplots(data_groups,
                       max_col=3,
                       title='Phase 5 - Metrics - Ivis - Binary',
                       out_path=OUT_PATH_GRAPH,
                       subplot_titles=['Learning Rate', 'Loss', 'Accuracy', 'AUC', 'Precision', 'Recall'])

# Phase 6 - Classification
- Sparse categorical crossentropy features

In [ ]:
# Split train & validation dataset with balanced target label
train_df = balanced_target(pd.concat([X_train_sparse, y_train], axis=1),
                           target=TARGET, n_remain=350, random_state=10000)
valid_df = balanced_target(pd.concat([X_train_sparse, y_train], axis=1),
                           target=TARGET, n_remain=150, random_state=10000, excludes=train_df.index)

# Shuffle dataset
train_df = train_df.sample(frac=1, random_state=0).reset_index(drop=True)
valid_df = valid_df.sample(frac=1, random_state=0).reset_index(drop=True)

train_df.shape, valid_df.shape

In [ ]:
print('Train dataset:')
vp.value_count(train_df, TARGET)

print('\nValidate dataset:')
vp.value_count(valid_df, TARGET)

In [ ]:
# Separate features & target
X_train_sparse, y_train_sparse = feature_target_split(train_df)
X_valid_sparse, y_valid_sparse = feature_target_split(valid_df)

# Categorical crossentropy target
y_train_sparse = target_onehot_encoder.transform(y_train_sparse.to_frame())
y_valid_sparse = target_onehot_encoder.transform(y_valid_sparse.to_frame())
y_test_sparse  = target_onehot_encoder.transform(y_test.to_frame())

print('Train dataset:')
print(X_train_sparse.shape, y_train_sparse.shape)

print('\nValidate dataset:')
print(X_valid_sparse.shape, y_valid_sparse.shape)

In [ ]:
# Feature scaling
minmax_scaler  = DFMinMaxScaler()
X_train_sparse = minmax_scaler.fit_transform(X_train_sparse)
X_valid_sparse = minmax_scaler.transform(X_valid_sparse)
X_test_sparse  = minmax_scaler.transform(X_test_sparse)

X_train_sparse.shape, X_valid_sparse.shape, X_test_sparse.shape

In [ ]:
# Reference: https://machinelearningmastery.com/understand-the-dynamics-of-learning-rate-on-deep-learning-neural-networks/
lrate = ReduceLROnPlateau(monitor='val_auc', factor=0.95, patience=15)

# Reference: https://machinelearningmastery.com/how-to-stop-training-deep-neural-networks-at-the-right-time-using-early-stopping/
es = EarlyStopping(monitor='val_auc', mode='max', verbose=1, patience=30, restore_best_weights=True)

model   = compile_model(X_train_sparse)
history = model.fit(X_train_sparse, y_train_sparse,
                    validation_data=(X_valid_sparse, y_valid_sparse),
                    epochs=100,
                    callbacks=[lrate, es],
                    batch_size=32)

In [ ]:
# Test set evaluation
eval_classif(
    np.argmax(y_test_sparse.values, axis=1),
    np.argmax(model.predict(X_test_sparse), axis=1)
)

In [ ]:
# Train set evaluation
eval_classif(
    np.argmax(y_train_sparse.values, axis=1),
    np.argmax(model.predict(X_train_sparse), axis=1)
)

In [ ]:
# Validation set evaluation
eval_classif(
    np.argmax(y_valid_sparse.values, axis=1),
    np.argmax(model.predict(X_valid_sparse), axis=1)
)

###### Line

In [ ]:
# Learning Rate
data = []
data.append(go.Scatter(
    y=history.history['lr'],
    mode='lines',
    name='LR',
    marker={'color': DEFAULT_PLOTLY_COLORS[-1]},
))
fig1 = go.Figure(data=data)

# Loss
data = []
data.append(go.Scatter(
    y=history.history['loss'],
    mode='lines',
    name='loss',
    marker={'color': DEFAULT_PLOTLY_COLORS[0]},
    legendgroup='train',
))
data.append(go.Scatter(
    y=history.history['val_loss'],
    mode='lines',
    name='val_loss',
    marker={'color': DEFAULT_PLOTLY_COLORS[1]},
    legendgroup='validate',
))
fig2 = go.Figure(data=data)

# Accuracy
data = []
data.append(go.Scattergl(
    y=history.history['categorical_accuracy'],
    mode='lines',
    name='accuracy',
    marker={'color': DEFAULT_PLOTLY_COLORS[0]},
    legendgroup='train',
))
data.append(go.Scattergl(
    y=history.history['val_categorical_accuracy'],
    mode='lines',
    name='val_accuracy',
    marker={'color': DEFAULT_PLOTLY_COLORS[1]},
    legendgroup='validate',
))
fig3 = go.Figure(data=data)

# AUC
data = []
data.append(go.Scattergl(
    y=history.history['auc'],
    mode='lines',
    name='auc',
    marker={'color': DEFAULT_PLOTLY_COLORS[0]},
    legendgroup='train',
))
data.append(go.Scattergl(
    y=history.history['val_auc'],
    mode='lines',
    name='val_auc',
    marker={'color': DEFAULT_PLOTLY_COLORS[1]},
    legendgroup='validate',
))
fig4 = go.Figure(data=data)

# Precision
data = []
data.append(go.Scatter(
    y=history.history['precision'],
    mode='lines',
    name='precision',
    marker={'color': DEFAULT_PLOTLY_COLORS[0]},
    legendgroup='train',
))
data.append(go.Scatter(
    y=history.history['val_precision'],
    mode='lines',
    name='val_precision',
    marker={'color': DEFAULT_PLOTLY_COLORS[1]},
    legendgroup='validate',
))
fig5 = go.Figure(data=data)

# Recall
data = []
data.append(go.Scatter(
    y=history.history['recall'],
    mode='lines',
    name='recall',
    marker={'color': DEFAULT_PLOTLY_COLORS[0]},
    legendgroup='train',
))
data.append(go.Scatter(
    y=history.history['val_recall'],
    mode='lines',
    name='val_recall',
    marker={'color': DEFAULT_PLOTLY_COLORS[1]},
    legendgroup='validate',
))
fig6 = go.Figure(data=data)

data_groups = [fig1['data'], fig2['data'], fig3['data'], fig4['data'], fig5['data'], fig6['data']]
vp.datagroups_subplots(data_groups,
                       max_col=3,
                       title='Phase 6 - Metrics - Ivis - Sparse Categorical',
                       out_path=OUT_PATH_GRAPH,
                       subplot_titles=['Learning Rate', 'Loss', 'Accuracy', 'AUC', 'Precision', 'Recall'])

# Phase 7 - Classification
- Binary + Sparse categorical crossentropy features

In [ ]:
X_train_combine = pd.concat([
    X_train_binary.rename(columns={'ivis_0': 'ivis_binary_0', 'ivis_1': 'ivis_binary_1'}),
    X_train_sparse.rename(columns={'ivis_0': 'ivis_sparse_0', 'ivis_1': 'ivis_sparse_1'}),
], axis=1)

X_valid_combine = pd.concat([
    X_valid_binary.rename(columns={'ivis_0': 'ivis_binary_0', 'ivis_1': 'ivis_binary_1'}),
    X_valid_sparse.rename(columns={'ivis_0': 'ivis_sparse_0', 'ivis_1': 'ivis_sparse_1'}),
], axis=1)

X_test_combine  = pd.concat([
    X_test_binary.rename(columns={'ivis_0': 'ivis_binary_0', 'ivis_1': 'ivis_binary_1'}),
    X_test_sparse.rename(columns={'ivis_0': 'ivis_sparse_0', 'ivis_1': 'ivis_sparse_1'}),
], axis=1)

X_train_combine.shape, X_valid_combine.shape, X_test_combine.shape

In [ ]:
y_train_combine = y_train_binary.copy()
y_valid_combine = y_valid_binary.copy()
y_test_combine  = y_test_binary.copy()

y_train_combine.shape, y_valid_combine.shape, y_test_combine.shape

In [ ]:
# Reference: https://machinelearningmastery.com/understand-the-dynamics-of-learning-rate-on-deep-learning-neural-networks/
lrate = ReduceLROnPlateau(monitor='val_auc', factor=0.95, patience=15)

# Reference: https://machinelearningmastery.com/how-to-stop-training-deep-neural-networks-at-the-right-time-using-early-stopping/
es = EarlyStopping(monitor='val_auc', mode='max', verbose=1, patience=30, restore_best_weights=True)

model   = compile_model(X_train_combine)
history = model.fit(X_train_combine, y_train_combine,
                    validation_data=(X_valid_combine, y_valid_combine),
                    epochs=100,
                    callbacks=[lrate, es],
                    batch_size=32)

In [ ]:
# Test set evaluation
eval_classif(
    np.argmax(y_test_combine.values, axis=1),
    np.argmax(model.predict(X_test_combine), axis=1)
)

In [ ]:
# Train set evaluation
eval_classif(
    np.argmax(y_train_combine.values, axis=1),
    np.argmax(model.predict(X_train_combine), axis=1)
)

In [ ]:
# Validation set evaluation
eval_classif(
    np.argmax(y_valid_combine.values, axis=1),
    np.argmax(model.predict(X_valid_combine), axis=1)
)

###### Line

In [ ]:
# Learning Rate
data = []
data.append(go.Scatter(
    y=history.history['lr'],
    mode='lines',
    name='LR',
    marker={'color': DEFAULT_PLOTLY_COLORS[-1]},
))
fig1 = go.Figure(data=data)

# Loss
data = []
data.append(go.Scatter(
    y=history.history['loss'],
    mode='lines',
    name='loss',
    marker={'color': DEFAULT_PLOTLY_COLORS[0]},
    legendgroup='train',
))
data.append(go.Scatter(
    y=history.history['val_loss'],
    mode='lines',
    name='val_loss',
    marker={'color': DEFAULT_PLOTLY_COLORS[1]},
    legendgroup='validate',
))
fig2 = go.Figure(data=data)

# Accuracy
data = []
data.append(go.Scattergl(
    y=history.history['categorical_accuracy'],
    mode='lines',
    name='accuracy',
    marker={'color': DEFAULT_PLOTLY_COLORS[0]},
    legendgroup='train',
))
data.append(go.Scattergl(
    y=history.history['val_categorical_accuracy'],
    mode='lines',
    name='val_accuracy',
    marker={'color': DEFAULT_PLOTLY_COLORS[1]},
    legendgroup='validate',
))
fig3 = go.Figure(data=data)

# AUC
data = []
data.append(go.Scattergl(
    y=history.history['auc'],
    mode='lines',
    name='auc',
    marker={'color': DEFAULT_PLOTLY_COLORS[0]},
    legendgroup='train',
))
data.append(go.Scattergl(
    y=history.history['val_auc'],
    mode='lines',
    name='val_auc',
    marker={'color': DEFAULT_PLOTLY_COLORS[1]},
    legendgroup='validate',
))
fig4 = go.Figure(data=data)

# Precision
data = []
data.append(go.Scatter(
    y=history.history['precision'],
    mode='lines',
    name='precision',
    marker={'color': DEFAULT_PLOTLY_COLORS[0]},
    legendgroup='train',
))
data.append(go.Scatter(
    y=history.history['val_precision'],
    mode='lines',
    name='val_precision',
    marker={'color': DEFAULT_PLOTLY_COLORS[1]},
    legendgroup='validate',
))
fig5 = go.Figure(data=data)

# Recall
data = []
data.append(go.Scatter(
    y=history.history['recall'],
    mode='lines',
    name='recall',
    marker={'color': DEFAULT_PLOTLY_COLORS[0]},
    legendgroup='train',
))
data.append(go.Scatter(
    y=history.history['val_recall'],
    mode='lines',
    name='val_recall',
    marker={'color': DEFAULT_PLOTLY_COLORS[1]},
    legendgroup='validate',
))
fig6 = go.Figure(data=data)

data_groups = [fig1['data'], fig2['data'], fig3['data'], fig4['data'], fig5['data'], fig6['data']]
vp.datagroups_subplots(data_groups,
                       max_col=3,
                       title='Phase 7 - Metrics - Ivis - Binary + Sparse Categorical',
                       out_path=OUT_PATH_GRAPH,
                       subplot_titles=['Learning Rate', 'Loss', 'Accuracy', 'AUC', 'Precision', 'Recall'])